In [1]:
# ============================================================
# Machine Learning Training Script
# Predicts "Time to Return" and "Span" using selected fields
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# === Step 1: Load data ===
file_path = "/kaggle/input/proserv-360-uiu-conbined-dataset-kader-and-kabir/Prediction Data.xlsx"
excel = pd.ExcelFile(file_path)
sheet_name = excel.sheet_names[0]
print(f"Loaded workbook: {file_path}")
print(f"Using sheet: {sheet_name}")

df = pd.read_excel(file_path, sheet_name=sheet_name)

# Ordered fields (1-based index)
ordered_fields = [
    'Parts Y/N', 'Service Region', 'Opened', 'Initial Closed Date', 'Month',
    'Span', 'Owner', 'Owner Name', 'Type', 'LCT', 'Zone', 'Entitlement',
    'Asset Number', 'Equipment Return to Customer', 'Time to Return', 'SR Type',
    'Status', 'Description', 'Problem Category', 'Modality', 'MOD',
    'Customer Equipment Status', 'Product Desc', 'Customer Name', 'Customer #',
    'System Id', 'Product', 'Model', 'Processed', 'Current Equipment Status',
    'Ship To Country', 'SR Sub Type', 'Closed', 'Source Sub Type'
]

# Input fields (20, 19, 23, 10, 1)
input_positions = [20, 19, 23, 10, 1]
X_cols = [ordered_fields[i-1] for i in input_positions]

# Target fields (15, 6)
target_positions = [15, 6]
y_cols = [ordered_fields[i-1] for i in target_positions]

print("\n=== Step 2: Select features and targets ===")
print(f"Feature columns: {X_cols}")
print(f"Target columns: {y_cols}")

# Keep only necessary columns
df_small = df[X_cols + y_cols].copy()

# === Step 3: Basic cleaning ===
for c in y_cols:
    df_small[c] = pd.to_numeric(df_small[c], errors='coerce')

before_rows = len(df_small)
df_small = df_small.dropna(subset=y_cols)
after_rows = len(df_small)
print(f"Dropped rows with missing targets: {before_rows - after_rows}")

X = df_small[X_cols]
y = df_small[y_cols]

# === Step 4: Build preprocessing & model ===
cat_features = list(range(len(X_cols)))

preprocess = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown='ignore', sparse=False), cat_features)]
)

base_model = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
model = Pipeline([
    ("preprocess", preprocess),
    ("regressor", MultiOutputRegressor(base_model))
])

print("\nPipeline created successfully!")

# === Step 5: Train/test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {X_train.shape[0]} rows, Test size: {X_test.shape[0]} rows")

# === Step 6: Train model ===
print("\nTraining model...")
model.fit(X_train, y_train)
print("Training complete.")

# === Step 7: Evaluate performance ===
y_pred = model.predict(X_test)

print("\n=== Test Performance ===")
for j, target in enumerate(y_cols):
    mae = mean_absolute_error(y_test.iloc[:, j], y_pred[:, j])
    rmse = mean_squared_error(y_test.iloc[:, j], y_pred[:, j], squared=False)
    r2 = r2_score(y_test.iloc[:, j], y_pred[:, j])
    print(f"{target}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

# === Step 8: Sample predictions ===
print("\nSample predictions (first 10 rows):")
sample = X_test.iloc[:10].copy()
sample[f"Actual {y_cols[0]}"] = y_test.iloc[:10, 0].values
sample[f"Pred {y_cols[0]}"] = y_pred[:10, 0]
sample[f"Actual {y_cols[1]}"] = y_test.iloc[:10, 1].values
sample[f"Pred {y_cols[1]}"] = y_pred[:10, 1]
print(sample)


Loaded workbook: /kaggle/input/proserv-360-uiu-conbined-dataset-kader-and-kabir/Prediction Data.xlsx
Using sheet: Sheet1

=== Step 2: Select features and targets ===
Feature columns: ['Modality', 'Problem Category', 'Product Desc', 'LCT', 'Parts Y/N']
Target columns: ['Time to Return', 'Span']
Dropped rows with missing targets: 0

Pipeline created successfully!
Train size: 2794 rows, Test size: 699 rows

Training model...


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Training complete.

=== Test Performance ===
Time to Return: MAE=1.0243, RMSE=2.3395, R²=-0.0485
Span: MAE=4.8330, RMSE=7.8640, R²=-0.0679

Sample predictions (first 10 rows):
     Modality  Problem Category                   Product Desc         LCT  \
1945       CT             Other     REVOLUTION ACTS SVCT INDIA     Chennai   
3362       CT     Imaging Issue             REVOLUTION HD 2000      Cochin   
2649       CT     Imaging Issue   REVOLUTION ACTS SVCT Z4G4 IN        Pune   
1880       CT             Other     REVOLUTION ACTS SVCT INDIA   Bangalore   
2768       CT             Other   REVOLUTION ACTS SVCT Z4G4 IN     Lucknow   
785        CT             Other                   OPTIMA CT660   Bangalore   
3306       CT     Imaging Issue    REVOLUTION EVO 3.7 MID HINO  Chandigarh   
1283       CT     Imaging Issue          REVO MAXIMA MID INDIA      Cochin   
678        CT     Imaging Issue  OPTIMA 660 M40 GT1700 BEIJING   Ahmedabad   
3072       CT  Mechanical Issue    REVOLUTIO

In [2]:
# MTBF (Time-to-Next-Failure) end-to-end training script
# - Uses Asset_System_ID as the asset key
# - Builds time-to-next-failure labels with censor handling
# - Engineers rolling/snapshot features
# - Trains: (1) last-k baseline, (2) RandomForestRegressor (with imputers)
# - Evaluates with MAE/MedAE/RMSE on a time-based test split

import re
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
from sklearn.inspection import permutation_importance

# -----------------------
# 0) Load
# -----------------------
PATH = Path("/kaggle/input/proserv-360-uiu-conbined-dataset-kader-and-kabir/Minimized data.xlsx")  # change if needed
assert PATH.exists(), f"File not found: {PATH}"

df_raw = pd.read_excel(PATH)
print(f"[INFO] Loaded shape: {df_raw.shape}")
print("[INFO] Columns (first 25):", list(df_raw.columns)[:25])

# -----------------------
# 1) Column utilities & detection
# -----------------------
ASSET_COL = "Asset_System_ID"
assert ASSET_COL in df_raw.columns, f"Expected asset key '{ASSET_COL}' not found."

def coerce_datetime(s):
    try:
        return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    except Exception:
        return pd.to_datetime(s, errors="coerce")

def find_time_col(df):
    priority = [
        "SR_Open_Date", "SR_Open_Time", "SR_Start_Time", "SR_Create_Date", "SR_Opened_Date",
        "SR_Date", "SR_Open_Timestamp", "SR_Created_On", "SR_Creation_Time", "SR_Start_Date",
        "SR_Close_Date", "SR_Closed_Date", "SR_End_Time"
    ]
    for c in priority:
        if c in df.columns:
            return c
    candidates = [c for c in df.columns if re.search(r"(date|time|timestamp)", c, re.IGNORECASE)]
    return candidates[0] if candidates else None

df = df_raw.copy()

TIME_CAND = find_time_col(df)
if TIME_CAND is not None:
    df["_event_time"] = coerce_datetime(df[TIME_CAND])
    print(f"[INFO] Using timestamp column: {TIME_CAND}")
else:
    df["_event_time"] = pd.NaT
    print("[WARN] No clear timestamp column; will synthesize ordering per asset.")

# -----------------------
# 2) Failure flag (taxonomy + simple heuristics)
# -----------------------
def infer_failure_flag(frame):
    flag = pd.Series(False, index=frame.index)
    if "SR_Type" in frame.columns:
        flag = flag | frame["SR_Type"].astype(str).str.contains(r"(corrective|breakdown)", case=False, regex=True)
    if "Activity_type_STD_AGG" in frame.columns:
        flag = flag | frame["Activity_type_STD_AGG"].astype(str).str.contains(r"corrective", case=False, regex=True)

    if "SR_TTR" in frame.columns:
        not_pm = pd.Series(True, index=frame.index)
        text_cols = [c for c in ["SR_Type", "Activity_type_STD_AGG"] if c in frame.columns]
        if text_cols:
            text_join = frame[text_cols].astype(str).agg(" ".join, axis=1)
            not_pm = ~text_join.str.contains(r"(pm|preventive|installation|training|upgrade)", case=False, regex=True)
        flag = flag | ((pd.to_numeric(frame["SR_TTR"], errors="coerce") > 0) & not_pm.fillna(True))
    return flag.fillna(False)

df["is_failure"] = infer_failure_flag(df)

# -----------------------
# 3) Ensure per-asset ordering/time
# -----------------------
def synthesize_time_within_asset(g):
    g = g.copy().sort_values("_event_time")
    if g["_event_time"].notna().any():
        # Fill remaining NaTs, guarantee monotonicity
        filled = g["_event_time"].ffill().bfill()
        order = np.arange(len(g))
        g["_event_time"] = filled + pd.to_timedelta(order, unit="s")
        return g
    # All NaT → synthesize daily increments from either install or a fixed origin
    origin = pd.Timestamp("2015-01-01")
    if "Asset_Install_Date" in g.columns:
        inst = coerce_datetime(g["Asset_Install_Date"])
        if inst.notna().any():
            origin = inst.min()
    g = g.reset_index(drop=True)
    g["_event_time"] = origin + pd.to_timedelta(np.arange(len(g)), unit="D")
    return g

df = df.groupby(ASSET_COL, group_keys=False).apply(synthesize_time_within_asset)

# -----------------------
# 4) Labels: time-to-next-failure (days) + censoring
# -----------------------
def build_next_failure_labels(g):
    g = g.sort_values("_event_time").copy()
    fail_idx = g.index[g["is_failure"]].tolist()
    t_next = []
    for idx, row in g.iterrows():
        t_row = row["_event_time"]
        next_times = [g.loc[j, "_event_time"] for j in fail_idx if g.loc[j, "_event_time"] > t_row]
        t_next.append(min(next_times) if next_times else pd.NaT)
    g["t_next_failure"] = t_next
    g["ttf_days"] = (g["t_next_failure"] - g["_event_time"]).dt.total_seconds() / 86400.0
    g["censored"] = g["t_next_failure"].isna()
    return g

labeled = df.groupby(ASSET_COL, group_keys=False).apply(build_next_failure_labels)
print("[INFO] Label stats (incl. censored):")
print(labeled["ttf_days"].describe())

# Keep uncensored (simple regression target)
trainable = labeled[~labeled["censored"] & (labeled["ttf_days"] > 0)].copy()
print(f"[INFO] Trainable rows (uncensored): {len(trainable)}")

# -----------------------
# 5) Rolling features/snapshots
# -----------------------
def add_rolling_features(frame, windows_days=(30, 90, 180)):
    f = frame.sort_values([ASSET_COL, "_event_time"]).set_index("_event_time").copy()
    f["SR_TTR_num"] = pd.to_numeric(f.get("SR_TTR", np.nan), errors="coerce")

    # time since last failure (days) — robust implementation
    def time_since_last_failure(g):
        t_series = pd.Series(g.index, index=g.index)
        last_fail_time = t_series.where(g["is_failure"]).ffill()
        return (t_series - last_fail_time).dt.total_seconds() / 86400.0
    f["time_since_last_failure"] = f.groupby(ASSET_COL, group_keys=False).apply(time_since_last_failure)

    for W in windows_days:
        win = f"{W}D"
        key = f"win{W}"
        f[f"fail_count_{key}"] = f["is_failure"].groupby(f[ASSET_COL]).rolling(win).sum().reset_index(level=0, drop=True)
        f[f"mean_ttr_{key}"] = f["SR_TTR_num"].groupby(f[ASSET_COL]).rolling(win).mean().reset_index(level=0, drop=True)
        f[f"var_ttr_{key}"]   = f["SR_TTR_num"].groupby(f[ASSET_COL]).rolling(win).var().reset_index(level=0, drop=True)
        f[f"sr_count_{key}"]  = pd.Series(1, index=f.index).groupby(f[ASSET_COL]).rolling(win).sum().reset_index(level=0, drop=True)

    f = f.reset_index()

    if "Asset_Install_Date" in f.columns:
        inst = coerce_datetime(f["Asset_Install_Date"])
        f["asset_age_days"] = (f["_event_time"] - inst).dt.total_seconds() / 86400.0
    else:
        f["asset_age_days"] = np.nan

    return f

feat = add_rolling_features(labeled)

# Merge features with uncensored labels
merged = feat.merge(trainable[[ASSET_COL, "_event_time", "ttf_days"]],
                    on=[ASSET_COL, "_event_time"], how="inner")

# If duplicate ttf columns appear, coalesce to a single 'ttf_days'
if "ttf_days" not in merged.columns:
    if "ttf_days_x" in merged.columns or "ttf_days_y" in merged.columns:
        merged["ttf_days"] = merged.get("ttf_days_x", np.nan).fillna(merged.get("ttf_days_y", np.nan))
        merged = merged.drop(columns=[c for c in ["ttf_days_x", "ttf_days_y"] if c in merged.columns])

train_df = merged
print("[INFO] Final training table shape:", train_df.shape)

# -----------------------
# 6) Feature lists
# -----------------------
categorical_cols = [c for c in [
    "OPH_Modality", "OPH_Product_Line", "OPH_Family_Name",
    "OPH_Planning_Name", "OPH_Product_Group", "OPH_PSI_Description",
    "OPH_Segment", "OPH_Product_Set_Desc", "City", "asset_installed_city", "site_name",
    "SR_Type", "Activity_type_STD_AGG"
] if c in train_df.columns]

numeric_cols = [c for c in train_df.columns if re.match(
    r"(fail_count_|mean_ttr_|var_ttr_|sr_count_|asset_age_days|time_since_last_failure)$", c) or c in ["SR_TTR_num"]
]

y_col = "ttf_days"
X_cols = categorical_cols + numeric_cols
assert y_col in train_df.columns, "ttf_days not found after merge."

# -----------------------
# 7) Time-based split (70/15/15 by unique event times)
# -----------------------
times_sorted = np.sort(train_df["_event_time"].unique())
cut1 = int(0.7 * len(times_sorted))
cut2 = int(0.85 * len(times_sorted))
t_train_max = times_sorted[cut1] if len(times_sorted) else pd.Timestamp("2022-01-01")
t_valid_max = times_sorted[cut2] if len(times_sorted) else pd.Timestamp("2023-01-01")

def split_by_time(frame):
    tr = frame[frame["_event_time"] <= t_train_max]
    va = frame[(frame["_event_time"] > t_train_max) & (frame["_event_time"] <= t_valid_max)]
    te = frame[frame["_event_time"] > t_valid_max]
    return tr, va, te

train_split, valid_split, test_split = split_by_time(train_df)
print("[INFO] Split sizes:", len(train_split), len(valid_split), len(test_split))

# -----------------------
# 8) Baseline: last-k (k=3) mean of prior inter-failure gaps per asset
# -----------------------
def compute_last_k_baseline(df_all, k=3):
    hist = df_all.sort_values([ASSET_COL, "_event_time"]).copy()
    ttf = hist.groupby(ASSET_COL)["ttf_days"].apply(lambda s: s.shift().rolling(k, min_periods=1).mean())
    hist["asset_ttf_hist"] = ttf.reset_index(level=0, drop=True)
    return hist["asset_ttf_hist"]

baseline_all = pd.concat([train_split, valid_split, test_split], axis=0).sort_values([ASSET_COL, "_event_time"])
baseline_all["baseline_pred"] = compute_last_k_baseline(baseline_all, k=3)

global_mean = train_split[y_col].mean() if len(train_split) else train_df[y_col].mean()
test_baseline = baseline_all.loc[test_split.index, "baseline_pred"].fillna(global_mean)

baseline_mae   = mean_absolute_error(test_split[y_col], test_baseline) if len(test_split) else np.nan
baseline_medae = median_absolute_error(test_split[y_col], test_baseline) if len(test_split) else np.nan
baseline_rmse  = mean_squared_error(test_split[y_col], test_baseline, squared=False) if len(test_split) else np.nan

print("\n[BASELINE] Last-interval mean (k=3)")
print(f"MAE:   {baseline_mae:,.2f} days" if pd.notna(baseline_mae) else "MAE: N/A")
print(f"MedAE: {baseline_medae:,.2f} days" if pd.notna(baseline_medae) else "MedAE: N/A")
print(f"RMSE:  {baseline_rmse:,.2f} days" if pd.notna(baseline_rmse) else "RMSE: N/A")

# -----------------------
# 9) Model: RandomForestRegressor with imputers + preprocessing
# -----------------------
categorical_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", min_frequency=10)),
])

numeric_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
])

preprocess = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("num", numeric_transformer, numeric_cols),
    ],
    remainder="drop"
)

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_split=4,
    min_samples_leaf=2,
    n_jobs=-1,
    random_state=42
)

model = Pipeline(steps=[("prep", preprocess), ("rf", rf)])

X_train = train_split[X_cols].copy()
y_train = train_split[y_col].values
X_valid = valid_split[X_cols].copy()
y_valid = valid_split[y_col].values
X_test  = test_split[X_cols].copy()
y_test  = test_split[y_col].values

print("\n[TRAIN] Fitting RandomForestRegressor with imputers...")
model.fit(pd.concat([X_train, X_valid]), np.concatenate([y_train, y_valid]))
print("[TRAIN] Done.")

pred_test = model.predict(X_test) if len(X_test) else np.array([])

if len(pred_test):
    mae   = mean_absolute_error(y_test, pred_test)
    medae = median_absolute_error(y_test, pred_test)
    rmse  = mean_squared_error(y_test, pred_test, squared=False)

    print("\n[TEST] RandomForestRegressor performance")
    print(f"MAE:   {mae:,.2f} days")
    print(f"MedAE: {medae:,.2f} days")
    print(f"RMSE:  {rmse:,.2f} days")

    # Optional: permutation importances (subset for speed)
    try:
        subset_n   = min(500, len(X_test))
        subset_idx = np.random.RandomState(42).choice(len(X_test), size=subset_n, replace=False)
        print("\n[EXPLAIN] Permutation importances (subset)")
        perm = permutation_importance(model, X_test.iloc[subset_idx], y_test[subset_idx],
                                      n_repeats=3, random_state=42, n_jobs=-1)

        # Reconstruct feature names
        cat_names = []
        if categorical_cols:
            cat_names = list(model.named_steps["prep"]
                                   .named_transformers_["cat"]
                                   .named_steps["ohe"]
                                   .get_feature_names_out(categorical_cols))
        num_names = numeric_cols
        all_feat_names = cat_names + num_names

        imp_df = pd.DataFrame({
            "feature": all_feat_names,
            "importance": perm.importances_mean
        }).sort_values("importance", ascending=False).head(20)

        print("\nTop 20 features by permutation importance:")
        for i, (f, v) in enumerate(zip(imp_df["feature"], imp_df["importance"]), 1):
            print(f"{i:>2}. {f:50s}  {v: .6f}")
    except Exception as e:
        print(f"[WARN] Permutation importance skipped: {e}")
else:
    print("\n[TEST] Not enough test rows to evaluate the model.")

print("\n[DONE] End-to-end MTBF (TTF) pipeline complete.")


[INFO] Loaded shape: (3119, 40)
[INFO] Columns (first 25): ['City', 'OPH_Modality', 'OPH_Product_Line', 'OPH_Segment', 'OPH_Family_Name', 'OPH_Planning_Name', 'OPH_Product_Group', 'OPH_PSI_Description', 'product_identifier', 'OPH_Product_Set_Desc', 'site_name', 'ucm_customer_id', 'asset_installed_city', 'SR_No', 'SR_Type_STD', 'SR_Owner_SSO', 'SR_Problem_Description', 'fcr_status', 'SR_Open_Date', 'SR_Close_Date', 'SR_Total_Labor_Hours', 'SR_Total_Travel_Hours', 'SR_Total_Part_Quantity', 'Asset_System_ID', 'Asset_Serial_Number']
[INFO] Using timestamp column: SR_Open_Date


/tmp/ipykernel_13/27277503.py:39: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_13/27277503.py:71: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  flag = flag | frame["SR_Type"].astype(str).str.contains(r"(corrective|breakdown)", case=False, regex=True)
/tmp/ipykernel_13/27277503.py:80: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  not_pm = ~text_join.str.contains(r"(pm|preventive|installation|training|upgrade)", case=False, regex=True)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:

[INFO] Label stats (incl. censored):
count    2833.000000
mean       80.476949
std       105.546355
min         0.000012
25%        15.003056
50%        39.230706
75%        99.385810
max       742.693426
Name: ttf_days, dtype: float64
[INFO] Trainable rows (uncensored): 2833


/tmp/ipykernel_13/27277503.py:39: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)


[INFO] Final training table shape: (2833, 60)
[INFO] Split sizes: 1990 422 421

[BASELINE] Last-interval mean (k=3)
MAE:   41.49 days
MedAE: 23.40 days
RMSE:  65.82 days

[TRAIN] Fitting RandomForestRegressor with imputers...
[TRAIN] Done.

[TEST] RandomForestRegressor performance
MAE:   53.59 days
MedAE: 39.41 days
RMSE:  72.80 days

[EXPLAIN] Permutation importances (subset)
[WARN] Permutation importance skipped: All arrays must be of the same length

[DONE] End-to-end MTBF (TTF) pipeline complete.


In [3]:
"""
MTBF-by-SystemID Model (Corrected)
- X: Asset_System_ID (string-normalized)
- Y: MTBF (expected inter-failure interval), learned from historical gaps
- Strategy:
    * If an asset has >=1 MTBF samples -> use per-asset MEAN (deterministic, per-ID)
    * Else fall back to cohort mean (Product Line / Modality / Family) -> else global mean
- Fixes:
    * Normalize Asset_System_ID to str in both fit & predict to avoid fallback for dtype mismatch
    * Compute EMA/last-K in true time order (using _event_time)
"""

import re
from pathlib import Path
from typing import Optional, Dict, Any, List
import numpy as np
import pandas as pd

# -----------------------
# Config
# -----------------------
EXCEL_PATH = Path("/kaggle/input/proserv-360-uiu-conbined-dataset-kader-and-kabir/Minimized data.xlsx")  # change if needed
ASSET_COL   = "Asset_System_ID"

# Cohort columns (ordered fallback preference)
COHORT_COLS = ["OPH_Product_Line", "OPH_Modality", "OPH_Family_Name"]

# Failure taxonomy (adjust if you have a clean SR_Type list)
FAILURE_REGEX     = r"(corrective|breakdown)"
NON_FAILURE_REGEX = r"(pm|preventive|installation|training|upgrade)"  # to exclude non-failures in heuristic

# -----------------------
# Utilities
# -----------------------
def coerce_datetime(s: pd.Series) -> pd.Series:
    try:
        return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    except Exception:
        return pd.to_datetime(s, errors="coerce")

def find_time_col(df: pd.DataFrame) -> Optional[str]:
    priority = [
        "SR_Open_Date", "SR_Open_Time", "SR_Start_Time", "SR_Create_Date", "SR_Opened_Date",
        "SR_Date", "SR_Open_Timestamp", "SR_Created_On", "SR_Creation_Time", "SR_Start_Date",
        "SR_Close_Date", "SR_Closed_Date", "SR_End_Time",
    ]
    for c in priority:
        if c in df.columns:
            return c
    # fallback: first date/time-like column
    for c in df.columns:
        if re.search(r"(date|time|timestamp)", str(c), re.IGNORECASE):
            return c
    return None

def infer_failure_flag(df: pd.DataFrame) -> pd.Series:
    """Mark service-impacting failures (heuristics + declared types)."""
    flag = pd.Series(False, index=df.index)
    if "SR_Type" in df.columns:
        flag = flag | df["SR_Type"].astype(str).str.contains(FAILURE_REGEX, case=False, regex=True)
    if "Activity_type_STD_AGG" in df.columns:
        flag = flag | df["Activity_type_STD_AGG"].astype(str).str.contains("corrective", case=False, regex=True)

    if "SR_TTR" in df.columns:
        text_cols = [c for c in ["SR_Type", "Activity_type_STD_AGG"] if c in df.columns]
        if text_cols:
            text_join = df[text_cols].astype(str).agg(" ".join, axis=1)
            not_pm = ~text_join.str.contains(NON_FAILURE_REGEX, case=False, regex=True)
        else:
            not_pm = True
        flag = flag | ((pd.to_numeric(df["SR_TTR"], errors="coerce") > 0) & pd.Series(not_pm, index=df.index))
    return flag.fillna(False)

def ensure_event_time(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure each asset has a monotone event time to order events, and normalize System ID to str."""
    df = df.copy()
    # normalize System ID to str early
    df[ASSET_COL] = df[ASSET_COL].astype(str)

    tcol = find_time_col(df)
    if tcol:
        df["_event_time"] = coerce_datetime(df[tcol])
    else:
        df["_event_time"] = pd.NaT

    def per_asset(g: pd.DataFrame) -> pd.DataFrame:
        g = g.sort_values("_event_time").copy()
        if g["_event_time"].notna().any():
            filled = g["_event_time"].ffill().bfill()
            order  = np.arange(len(g))
            g["_event_time"] = filled + pd.to_timedelta(order, unit="s")
            return g
        # all NaT => synthesize from install date or a fixed origin
        origin = pd.Timestamp("2015-01-01")
        if "Asset_Install_Date" in g.columns:
            inst = coerce_datetime(g["Asset_Install_Date"])
            if inst.notna().any():
                origin = inst.min()
        g = g.reset_index(drop=True)
        g["_event_time"] = origin + pd.to_timedelta(np.arange(len(g)), unit="D")
        return g

    return df.groupby(ASSET_COL, group_keys=False).apply(per_asset)

def build_inter_failure_intervals(df: pd.DataFrame) -> pd.DataFrame:
    """Compute inter-failure intervals (days) for each asset from consecutive failure events."""
    df = df.copy()
    df["is_failure"] = infer_failure_flag(df)
    df = df.sort_values([ASSET_COL, "_event_time"])

    def asset_gaps(g: pd.DataFrame) -> pd.DataFrame:
        g = g.sort_values("_event_time").copy()
        fail_times = g.loc[g["is_failure"], "_event_time"]
        if len(fail_times) < 2:
            g["mtbf_sample_days"] = np.nan
            return g
        gaps = (fail_times.shift(-1) - fail_times).dt.total_seconds() / 86400.0
        g["mtbf_sample_days"] = np.nan
        g.loc[fail_times.index[:-1], "mtbf_sample_days"] = gaps.iloc[:-1].values
        return g

    return df.groupby(ASSET_COL, group_keys=False).apply(asset_gaps)

def exp_moving_average(sequence: pd.Series, alpha: float = 0.5) -> float:
    vals = pd.to_numeric(sequence, errors="coerce").dropna().values
    if len(vals) == 0:
        return np.nan
    ema = vals[0]
    for v in vals[1:]:
        ema = alpha * v + (1 - alpha) * ema
    return float(ema)

# -----------------------
# Model
# -----------------------
class MTBFModel:
    """
    Lookup-style MTBF model per System ID (string-normalized).
      - If an asset has >=1 MTBF samples -> use per-asset mean (stable per-ID output)
      - Else -> cohort mean (Product Line / Modality / Family)
      - Else -> global mean
    """

    def __init__(self, cohort_cols: List[str] = COHORT_COLS):
        self.cohort_cols = list(cohort_cols)
        self.asset_stats: pd.DataFrame = pd.DataFrame()
        self.cohort_stats: Dict[str, pd.DataFrame] = {}
        self.global_mean: float = float("nan")

    def fit(self, df_raw: pd.DataFrame) -> "MTBFModel":
        assert ASSET_COL in df_raw.columns, f"Missing {ASSET_COL}"

        # Prepare & normalize types
        df = ensure_event_time(df_raw)
        df = build_inter_failure_intervals(df)

        # Collect MTBF samples with time & cohorts
        keep_cols = [ASSET_COL, "_event_time", "mtbf_sample_days"] + [c for c in self.cohort_cols if c in df.columns]
        samples = df.loc[~df["mtbf_sample_days"].isna(), keep_cols].copy()

        # Normalize System ID to string (in case ensure_event_time missed anything upstream)
        samples[ASSET_COL] = samples[ASSET_COL].astype(str)

        # Per-asset aggregates
        agg = (samples.groupby(ASSET_COL)["mtbf_sample_days"]
                      .agg(n_samples="count", asset_mean="mean", asset_median="median"))

        # EMA & last-3 in true time order
        ema_vals = []
        last3_vals = []
        for asset_id, g in samples.groupby(ASSET_COL):
            seq = g.sort_values("_event_time")["mtbf_sample_days"]
            ema_vals.append(exp_moving_average(seq, alpha=0.5))
            last3_vals.append(float(np.nanmean(seq.tail(3))) if len(seq) else np.nan)

        agg["asset_ema"]   = ema_vals
        agg["asset_last3"] = last3_vals

        # Ensure string index
        agg.index = agg.index.astype(str)
        self.asset_stats = agg

        # Cohort stats
        for col in self.cohort_cols:
            if col in samples.columns:
                tab = (samples.groupby(col)["mtbf_sample_days"]
                             .agg(n="count", mean="mean", median="median"))
                self.cohort_stats[col] = tab

        # Global mean
        self.global_mean = float(samples["mtbf_sample_days"].mean()) if len(samples) else float("nan")
        return self

    def _lookup_cohort_mean(self, df_row: pd.Series) -> Optional[float]:
        for col in self.cohort_cols:
            if col in df_row.index and col in self.cohort_stats and pd.notna(df_row[col]):
                ctab = self.cohort_stats[col]
                key = df_row[col]
                if key in ctab.index and pd.notna(ctab.loc[key, "mean"]):
                    return float(ctab.loc[key, "mean"])
        return None

    def predict_single(self, system_id: Any, df_context: Optional[pd.DataFrame] = None) -> Dict[str, Any]:
        sid = str(system_id)  # normalize
        out = {
            ASSET_COL: sid,
            "n_samples": 0,
            "asset_mean": None,
            "asset_ema": None,
            "asset_last3": None,
            "cohort_mean": None,
            "global_mean": self.global_mean,
            "prediction_days": None,
            "strategy": None,
        }

        # Per-asset stats (string index)
        if sid in self.asset_stats.index:
            row = self.asset_stats.loc[sid]
            out["n_samples"]  = int(row["n_samples"])
            out["asset_mean"] = (None if pd.isna(row["asset_mean"]) else float(row["asset_mean"]))
            out["asset_ema"]  = (None if pd.isna(row["asset_ema"]) else float(row["asset_ema"]))
            out["asset_last3"]= (None if pd.isna(row["asset_last3"]) else float(row["asset_last3"]))

        # Cohort fallback needs asset attributes
        if df_context is not None and out["cohort_mean"] is None:
            ctx = df_context.copy()
            ctx[ASSET_COL] = ctx[ASSET_COL].astype(str)
            rows = ctx.loc[ctx[ASSET_COL] == sid]
            if len(rows):
                latest = rows.sort_values("_event_time").iloc[-1]
                cmean = self._lookup_cohort_mean(latest)
                if cmean is not None:
                    out["cohort_mean"] = cmean

        # Prediction policy:
        # Prefer per-asset MEAN if any history exists (ensures per-ID differences)
        if out["n_samples"] >= 1 and out["asset_mean"] is not None:
            out["prediction_days"] = out["asset_mean"]
            out["strategy"] = "asset_mean(n>=1)"
        elif out["cohort_mean"] is not None:
            out["prediction_days"] = float(out["cohort_mean"])
            out["strategy"] = "cohort_mean"
        else:
            out["prediction_days"] = float(out["global_mean"])
            out["strategy"] = "global_mean"

        return out

    def predict(self, system_ids: List[Any], df_context: Optional[pd.DataFrame] = None) -> pd.DataFrame:
        rows = [self.predict_single(sid, df_context=df_context) for sid in system_ids]
        return pd.DataFrame(rows)

# -----------------------
# Train the model and demo predictions
# -----------------------
def main():
    assert EXCEL_PATH.exists(), f"File not found: {EXCEL_PATH}"
    df = pd.read_excel(EXCEL_PATH)

    # Prepare timeline & normalize IDs (adds _event_time)
    df = ensure_event_time(df)

    # Fit
    model = MTBFModel(cohort_cols=COHORT_COLS).fit(df)

    print("\n[MODEL] Learned statistics")
    print(f"  Global MTBF mean (days): {model.global_mean:.2f}" if not np.isnan(model.global_mean) else "  Global MTBF mean: NaN")
    print(f"  Assets with history (n_samples>=1): {int((model.asset_stats['n_samples'] >= 1).sum())}")

    for c in COHORT_COLS:
        if c in model.cohort_stats:
            print(f"  Cohort table: {c} (groups={len(model.cohort_stats[c])})")

    # Demo predictions for first 5 IDs in the data
    demo_ids = list(df[ASSET_COL].astype(str).dropna().unique()[:5])
    preds = model.predict(demo_ids, df_context=df)

    print("\n[PREDICTIONS] MTBF days by System ID (per-asset if available):")
    print(preds.to_string(index=False))

    # Example callable:
    # def predict_mtbf(system_id: str) -> float:
    #     return float(model.predict([system_id], df_context=df).iloc[0]["prediction_days"])

if __name__ == "__main__":
    main()

/tmp/ipykernel_13/2199906926.py:37: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_13/2199906926.py:103: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(ASSET_COL, group_keys=False).apply(per_asset)
/tmp/ipykernel_13/2199906926.py:37: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see


[MODEL] Learned statistics
  Global MTBF mean (days): 63.24
  Assets with history (n_samples>=1): 148
  Cohort table: OPH_Product_Line (groups=5)
  Cohort table: OPH_Modality (groups=5)
  Cohort table: OPH_Family_Name (groups=28)

[PREDICTIONS] MTBF days by System ID (per-asset if available):
Asset_System_ID  n_samples  asset_mean  asset_ema  asset_last3  cohort_mean  global_mean  prediction_days         strategy
 83021600005521          6   91.498127 128.780676   104.698549    82.933964    63.239357        91.498127 asset_mean(n>=1)
 83021600015312          6  133.976250 136.814831   125.957315    82.933964    63.239357       133.976250 asset_mean(n>=1)
 83021600098222         12   45.519664  49.761622    48.659726    82.933964    63.239357        45.519664 asset_mean(n>=1)
 83021600100022          4  155.065191  81.534463   204.000841    82.933964    63.239357       155.065191 asset_mean(n>=1)
 83021600103323         14   29.446710  30.349376    35.096377    82.933964    63.239357  

/tmp/ipykernel_13/2199906926.py:122: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(ASSET_COL, group_keys=False).apply(asset_gaps)


In [4]:
"""
MTBF-by-SystemID Model (Corrected)
- X: Asset_System_ID (string-normalized)
- Y: MTBF (expected inter-failure interval), learned from historical gaps
- Strategy:
    * If an asset has >=1 MTBF samples -> use per-asset MEAN (deterministic, per-ID)
    * Else fall back to cohort mean (Product Line / Modality / Family) -> else global mean
- Fixes:
    * Normalize Asset_System_ID to str in both fit & predict to avoid fallback for dtype mismatch
    * Compute EMA/last-K in true time order (using _event_time)
"""

import re
from pathlib import Path
from typing import Optional, Dict, Any, List
import numpy as np
import pandas as pd

# -----------------------
# Config
# -----------------------
EXCEL_PATH = Path("/kaggle/input/proserv-360-uiu-conbined-dataset-kader-and-kabir/Minimized data.xlsx")  # change if needed
ASSET_COL   = "Asset_System_ID"

# Cohort columns (ordered fallback preference)
COHORT_COLS = ["OPH_Product_Line", "OPH_Modality", "OPH_Family_Name"]

# Failure taxonomy (adjust if you have a clean SR_Type list)
FAILURE_REGEX     = r"(corrective|breakdown)"
NON_FAILURE_REGEX = r"(pm|preventive|installation|training|upgrade)"  # to exclude non-failures in heuristic

# -----------------------
# Utilities
# -----------------------
def coerce_datetime(s: pd.Series) -> pd.Series:
    try:
        return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    except Exception:
        return pd.to_datetime(s, errors="coerce")

def find_time_col(df: pd.DataFrame) -> Optional[str]:
    priority = [
        "SR_Open_Date", "SR_Open_Time", "SR_Start_Time", "SR_Create_Date", "SR_Opened_Date",
        "SR_Date", "SR_Open_Timestamp", "SR_Created_On", "SR_Creation_Time", "SR_Start_Date",
        "SR_Close_Date", "SR_Closed_Date", "SR_End_Time",
    ]
    for c in priority:
        if c in df.columns:
            return c
    # fallback: first date/time-like column
    for c in df.columns:
        if re.search(r"(date|time|timestamp)", str(c), re.IGNORECASE):
            return c
    return None

def infer_failure_flag(df: pd.DataFrame) -> pd.Series:
    """Mark service-impacting failures (heuristics + declared types)."""
    flag = pd.Series(False, index=df.index)
    if "SR_Type" in df.columns:
        flag = flag | df["SR_Type"].astype(str).str.contains(FAILURE_REGEX, case=False, regex=True)
    if "Activity_type_STD_AGG" in df.columns:
        flag = flag | df["Activity_type_STD_AGG"].astype(str).str.contains("corrective", case=False, regex=True)

    if "SR_TTR" in df.columns:
        text_cols = [c for c in ["SR_Type", "Activity_type_STD_AGG"] if c in df.columns]
        if text_cols:
            text_join = df[text_cols].astype(str).agg(" ".join, axis=1)
            not_pm = ~text_join.str.contains(NON_FAILURE_REGEX, case=False, regex=True)
        else:
            not_pm = True
        flag = flag | ((pd.to_numeric(df["SR_TTR"], errors="coerce") > 0) & pd.Series(not_pm, index=df.index))
    return flag.fillna(False)

def ensure_event_time(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure each asset has a monotone event time to order events, and normalize System ID to str."""
    df = df.copy()
    # normalize System ID to str early
    df[ASSET_COL] = df[ASSET_COL].astype(str)

    tcol = find_time_col(df)
    if tcol:
        df["_event_time"] = coerce_datetime(df[tcol])
    else:
        df["_event_time"] = pd.NaT

    def per_asset(g: pd.DataFrame) -> pd.DataFrame:
        g = g.sort_values("_event_time").copy()
        if g["_event_time"].notna().any():
            filled = g["_event_time"].ffill().bfill()
            order  = np.arange(len(g))
            g["_event_time"] = filled + pd.to_timedelta(order, unit="s")
            return g
        # all NaT => synthesize from install date or a fixed origin
        origin = pd.Timestamp("2015-01-01")
        if "Asset_Install_Date" in g.columns:
            inst = coerce_datetime(g["Asset_Install_Date"])
            if inst.notna().any():
                origin = inst.min()
        g = g.reset_index(drop=True)
        g["_event_time"] = origin + pd.to_timedelta(np.arange(len(g)), unit="D")
        return g

    return df.groupby(ASSET_COL, group_keys=False).apply(per_asset)

def build_inter_failure_intervals(df: pd.DataFrame) -> pd.DataFrame:
    """Compute inter-failure intervals (days) for each asset from consecutive failure events."""
    df = df.copy()
    df["is_failure"] = infer_failure_flag(df)
    df = df.sort_values([ASSET_COL, "_event_time"])

    def asset_gaps(g: pd.DataFrame) -> pd.DataFrame:
        g = g.sort_values("_event_time").copy()
        fail_times = g.loc[g["is_failure"], "_event_time"]
        if len(fail_times) < 2:
            g["mtbf_sample_days"] = np.nan
            return g
        gaps = (fail_times.shift(-1) - fail_times).dt.total_seconds() / 86400.0
        g["mtbf_sample_days"] = np.nan
        g.loc[fail_times.index[:-1], "mtbf_sample_days"] = gaps.iloc[:-1].values
        return g

    return df.groupby(ASSET_COL, group_keys=False).apply(asset_gaps)

def exp_moving_average(sequence: pd.Series, alpha: float = 0.5) -> float:
    vals = pd.to_numeric(sequence, errors="coerce").dropna().values
    if len(vals) == 0:
        return np.nan
    ema = vals[0]
    for v in vals[1:]:
        ema = alpha * v + (1 - alpha) * ema
    return float(ema)

# -----------------------
# Model
# -----------------------
class MTBFModel:
    """
    Lookup-style MTBF model per System ID (string-normalized).
      - If an asset has >=1 MTBF samples -> use per-asset mean (stable per-ID output)
      - Else -> cohort mean (Product Line / Modality / Family)
      - Else -> global mean
    """

    def __init__(self, cohort_cols: List[str] = COHORT_COLS):
        self.cohort_cols = list(cohort_cols)
        self.asset_stats: pd.DataFrame = pd.DataFrame()
        self.cohort_stats: Dict[str, pd.DataFrame] = {}
        self.global_mean: float = float("nan")

    def fit(self, df_raw: pd.DataFrame) -> "MTBFModel":
        assert ASSET_COL in df_raw.columns, f"Missing {ASSET_COL}"

        # Prepare & normalize types
        df = ensure_event_time(df_raw)
        df = build_inter_failure_intervals(df)

        # Collect MTBF samples with time & cohorts
        keep_cols = [ASSET_COL, "_event_time", "mtbf_sample_days"] + [c for c in self.cohort_cols if c in df.columns]
        samples = df.loc[~df["mtbf_sample_days"].isna(), keep_cols].copy()

        # Normalize System ID to string (in case ensure_event_time missed anything upstream)
        samples[ASSET_COL] = samples[ASSET_COL].astype(str)

        # Per-asset aggregates
        agg = (samples.groupby(ASSET_COL)["mtbf_sample_days"]
                      .agg(n_samples="count", asset_mean="mean", asset_median="median"))

        # EMA & last-3 in true time order
        ema_vals = []
        last3_vals = []
        for asset_id, g in samples.groupby(ASSET_COL):
            seq = g.sort_values("_event_time")["mtbf_sample_days"]
            ema_vals.append(exp_moving_average(seq, alpha=0.5))
            last3_vals.append(float(np.nanmean(seq.tail(3))) if len(seq) else np.nan)

        agg["asset_ema"]   = ema_vals
        agg["asset_last3"] = last3_vals

        # Ensure string index
        agg.index = agg.index.astype(str)
        self.asset_stats = agg

        # Cohort stats
        for col in self.cohort_cols:
            if col in samples.columns:
                tab = (samples.groupby(col)["mtbf_sample_days"]
                             .agg(n="count", mean="mean", median="median"))
                self.cohort_stats[col] = tab

        # Global mean
        self.global_mean = float(samples["mtbf_sample_days"].mean()) if len(samples) else float("nan")
        return self

    def _lookup_cohort_mean(self, df_row: pd.Series) -> Optional[float]:
        for col in self.cohort_cols:
            if col in df_row.index and col in self.cohort_stats and pd.notna(df_row[col]):
                ctab = self.cohort_stats[col]
                key = df_row[col]
                if key in ctab.index and pd.notna(ctab.loc[key, "mean"]):
                    return float(ctab.loc[key, "mean"])
        return None

    def predict_single(self, system_id: Any, df_context: Optional[pd.DataFrame] = None) -> Dict[str, Any]:
        sid = str(system_id)  # normalize
        out = {
            ASSET_COL: sid,
            "n_samples": 0,
            "asset_mean": None,
            "asset_ema": None,
            "asset_last3": None,
            "cohort_mean": None,
            "global_mean": self.global_mean,
            "prediction_days": None,
            "strategy": None,
        }

        # Per-asset stats (string index)
        if sid in self.asset_stats.index:
            row = self.asset_stats.loc[sid]
            out["n_samples"]  = int(row["n_samples"])
            out["asset_mean"] = (None if pd.isna(row["asset_mean"]) else float(row["asset_mean"]))
            out["asset_ema"]  = (None if pd.isna(row["asset_ema"]) else float(row["asset_ema"]))
            out["asset_last3"]= (None if pd.isna(row["asset_last3"]) else float(row["asset_last3"]))

        # Cohort fallback needs asset attributes
        if df_context is not None and out["cohort_mean"] is None:
            ctx = df_context.copy()
            ctx[ASSET_COL] = ctx[ASSET_COL].astype(str)
            rows = ctx.loc[ctx[ASSET_COL] == sid]
            if len(rows):
                latest = rows.sort_values("_event_time").iloc[-1]
                cmean = self._lookup_cohort_mean(latest)
                if cmean is not None:
                    out["cohort_mean"] = cmean

        # Prediction policy:
        # Prefer per-asset MEAN if any history exists (ensures per-ID differences)
        if out["n_samples"] >= 1 and out["asset_mean"] is not None:
            out["prediction_days"] = out["asset_mean"]
            out["strategy"] = "asset_mean(n>=1)"
        elif out["cohort_mean"] is not None:
            out["prediction_days"] = float(out["cohort_mean"])
            out["strategy"] = "cohort_mean"
        else:
            out["prediction_days"] = float(out["global_mean"])
            out["strategy"] = "global_mean"

        return out

    def predict(self, system_ids: List[Any], df_context: Optional[pd.DataFrame] = None) -> pd.DataFrame:
        rows = [self.predict_single(sid, df_context=df_context) for sid in system_ids]
        return pd.DataFrame(rows)

# -----------------------
# Train the model and demo predictions
# -----------------------
def main():
    assert EXCEL_PATH.exists(), f"File not found: {EXCEL_PATH}"
    df = pd.read_excel(EXCEL_PATH)

    # Prepare timeline & normalize IDs (adds _event_time)
    df = ensure_event_time(df)

    # Fit
    model = MTBFModel(cohort_cols=COHORT_COLS).fit(df)

    print("\n[MODEL] Learned statistics")
    print(f"  Global MTBF mean (days): {model.global_mean:.2f}" if not np.isnan(model.global_mean) else "  Global MTBF mean: NaN")
    print(f"  Assets with history (n_samples>=1): {int((model.asset_stats['n_samples'] >= 1).sum())}")

    for c in COHORT_COLS:
        if c in model.cohort_stats:
            print(f"  Cohort table: {c} (groups={len(model.cohort_stats[c])})")

    # Demo predictions for first 5 IDs in the data
    demo_ids = list(df[ASSET_COL].astype(str).dropna().unique()[:10])
    preds = model.predict(demo_ids, df_context=df)

    print("\n[PREDICTIONS] MTBF days by System ID (per-asset if available):")
    print(preds.to_string(index=False))

    #Example callable:
    def predict_mtbf(system_id: str) -> float:
        return float(model.predict([system_id], df_context=df).iloc[0]["prediction_days"])

if __name__ == "__main__":
    main()

/tmp/ipykernel_13/2980404613.py:37: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_13/2980404613.py:103: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(ASSET_COL, group_keys=False).apply(per_asset)
/tmp/ipykernel_13/2980404613.py:37: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see


[MODEL] Learned statistics
  Global MTBF mean (days): 63.24
  Assets with history (n_samples>=1): 148
  Cohort table: OPH_Product_Line (groups=5)
  Cohort table: OPH_Modality (groups=5)
  Cohort table: OPH_Family_Name (groups=28)

[PREDICTIONS] MTBF days by System ID (per-asset if available):
Asset_System_ID  n_samples  asset_mean  asset_ema  asset_last3  cohort_mean  global_mean  prediction_days         strategy
 83021600005521          6   91.498127 128.780676   104.698549    82.933964    63.239357        91.498127 asset_mean(n>=1)
 83021600015312          6  133.976250 136.814831   125.957315    82.933964    63.239357       133.976250 asset_mean(n>=1)
 83021600098222         12   45.519664  49.761622    48.659726    82.933964    63.239357        45.519664 asset_mean(n>=1)
 83021600100022          4  155.065191  81.534463   204.000841    82.933964    63.239357       155.065191 asset_mean(n>=1)
 83021600103323         14   29.446710  30.349376    35.096377    82.933964    63.239357  

/tmp/ipykernel_13/2980404613.py:122: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(ASSET_COL, group_keys=False).apply(asset_gaps)


In [5]:
def predict_mtbf(system_id: str) -> float:
        return float(model.predict([system_id], df_context=df).iloc[0]["prediction_days"])

In [6]:
ASSET_COL = "Asset_System_ID"

# Must exist from training:
# - model       (your sklearn Pipeline)
# - train_df    (engineered features table)
# - X_cols      (the exact list of feature columns used to train `model`)

def predict_mtbf_sklearn(system_id):
    sid = str(system_id).strip()

    # Pick the most recent feature snapshot for this asset
    rows = (train_df[train_df[ASSET_COL].astype(str) == sid]
            .sort_values("_event_time"))
    if rows.empty:
        raise ValueError(f"No engineered feature row found for System ID {sid}. "
                         f"Use the lookup model (`mtbf_model`) or compute features for this asset.")
    
    X = rows.iloc[[-1]][X_cols]   # keep as DataFrame with expected columns
    yhat = model.predict(X)[0]    # sklearn pipeline
    print("The MTBF is =", yhat, "days")
    pmint = 0.8*yhat
    print("Suggested PM Interval =", pmint, "days")
    return 



In [7]:
# Test
predict_mtbf_sklearn("8302160822967")

The MTBF is = 102.16887256547624 days
Suggested PM Interval = 81.73509805238099 days


In [8]:
# -----------------------
# Generate Excel Report for All System IDs
# -----------------------

def generate_mtbf_report():
    # Get all unique system IDs from the training data
    all_system_ids = train_df[ASSET_COL].astype(str).unique()
    print(f"[INFO] Generating report for {len(all_system_ids)} system IDs...")
    
    results = []
    
    for i, system_id in enumerate(all_system_ids):
        if i % 100 == 0:  # Progress indicator
            print(f"Processing {i}/{len(all_system_ids)}...")
        
        try:
            # Get the most recent feature snapshot for this asset
            rows = (train_df[train_df[ASSET_COL].astype(str) == system_id]
                    .sort_values("_event_time"))
            
            if not rows.empty:
                X = rows.iloc[[-1]][X_cols]   # Most recent record
                mtbf = model.predict(X)[0]    # sklearn pipeline prediction
                pm_interval = 0.8 * mtbf      # 80% of MTBF
                
                # Round to 2 decimal places
                mtbf_rounded = round(mtbf, 2)
                pm_interval_rounded = round(pm_interval, 2)
                
                results.append({
                    'System_ID': system_id,
                    'MTBF_days': mtbf_rounded,
                    'Suggested_PM_Interval_days': pm_interval_rounded
                })
            else:
                print(f"[WARN] No data found for System ID: {system_id}")
                
        except Exception as e:
            print(f"[ERROR] Failed to process System ID {system_id}: {e}")
            # You can choose to skip or add placeholder values
            results.append({
                'System_ID': system_id,
                'MTBF_days': 'Error',
                'Suggested_PM_Interval_days': 'Error'
            })
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Save to Excel
    output_file = "MTBF_Report_All_Systems.xlsx"
    results_df.to_excel(output_file, index=False)
    
    print(f"\n[SUCCESS] Report generated: {output_file}")
    print(f"Total records: {len(results_df)}")
    print("\nFirst 10 records:")
    print(results_df.head(10).to_string(index=False))
    
    return results_df

# Generate the report
mtbf_report = generate_mtbf_report()

[INFO] Generating report for 158 system IDs...
Processing 0/158...
Processing 100/158...

[SUCCESS] Report generated: MTBF_Report_All_Systems.xlsx
Total records: 158

First 10 records:
     System_ID  MTBF_days  Suggested_PM_Interval_days
 8302160822967     102.17                       81.74
83021600005521      60.35                       48.28
83021600015312     146.23                      116.99
83021600098222      39.37                       31.50
83021600100022      24.81                       19.84
83021600103323      29.09                       23.27
83021600108822      60.83                       48.66
83021600108923      13.02                       10.42
83021600184824      72.76                       58.21
83021600420416      99.81                       79.84
